# Frequent itemset mining

- Student ID: 21127021
- Student name: Truong Van Chi

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) #* Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) #* A python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #* Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    return data, s

### Tree Projection

**Question 0: I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**Answer:**
Apriori Algorithm sinh ra tất cả tập con ở level k+1 từ level k sau đó mới tiếp tục tìm kiếm các frequent itemset tại level k+1 (ý tưởng giống thuật toán BFS). Ngược lại Tree Projection tại node X ở level k sẽ sinh ra tất cả tập con có thể của level k+1 có chung tiền tố tại node X và xét level k+1 tiếp theo và cứ tiếp tục đi xâu xuống (ý tưởng tương tự thuật toán DFS). Hơn thế nữa, với thuật toán Apriori ta phải kiểm tra các item từ 1 đến k-1 sau đó mới có thể join 2 tập item lại để tạo ra itemset có k+1 item. Đối với Tree Projection ta lưu một cây tiền tố nên không cần kiểm tra điều kiện như trên mà vẫn có thể sinh ra được tập con tiếp theo.

In [3]:
def joinset(a,b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space) 

    -------------------
    return 
        ret: itemset generated by joining a and b
    '''
    
    return list(set().union(a, b))
    # a,b same brach -> a_k = b_k
    # Hint: this function will be called in generateSearchSpace method.:
    
    
class TP:
    def __init__ (self, data=None, s=None, minSup=None):
        self.data=data
        self.s={}
        
        #Sort support ascending
        for key, support in sorted(s.items(),key= lambda item: item[1]):
            self.s[key]=support
        # Why should we do this, answer it at the markdown below?
            
        self.minSup=minSup
        self.L={}  #Store frequent itemsets mined from database
        self.runAlgorithm()
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree={}

        search_space={}  
        for item, support in self.s.items():
            search_space[item]={}
            
            search_space[item]['itemset']=[item] 
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            
            search_space[item]['pruned']=False

            
            search_space[item]['support']=support
            
            tree[item]={}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.
            
            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            ''' 
        
        return tree, search_space
    
    def computeItemsetSupport(self, itemset):
        
        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for transaction in self.data:
            flag = True
            for item in itemset:
                if (item not in self.data[transaction]): flag = False
            if (flag): support += 1
       
        return support
    
    
        
    def prune(self,k, tree, search_space):
        
        '''
        In this method we will find out which itemset in current search space is frequent 
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets. 
        '''
        #TODO
        if k not in self.L:
            self.L[k] = []
            
        items = tree.keys()
        for item in items:
            if (search_space[item]['support'] < self.minSup):
                search_space[item]['pruned'] = True
            else:
                self.L[k].append(search_space[item]['itemset'])


       
    def generateSearchSpace(self,k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function) 
        '''
        items=list(tree.keys())  
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l=len(items)
        self.prune(k, tree, search_space)
        if l==0: return   #Stop condition
        
        for i in range(l-1):
            a=items[i]
            if search_space[a]['pruned']: continue 
            
            # Active item
            for j in range(i+1,l):
                b=items[j]
                
                search_space[a][b]={}
                tree[a][b]={}
                #TODO
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft). 
            
                #First create newset using join set
                joinSet = joinset(search_space[a]['itemset'], search_space[b]['itemset'])
                
                #Second add newset to search_space
                search_space[a][b]['itemset'] = joinSet
                search_space[a][b]['pruned'] = False
                search_space[a][b]['support'] = self.computeItemsetSupport(joinSet)
                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k+1,tree[a],search_space[a])

    
    def runAlgorithm(self):
        tree,search_space=self.initialize() #generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
    def miningResults(self):
        return self.L
        

Ok, let's test on a typical dataset `chess`.

In [4]:
transactions, freq= readData('chess.txt')

In [5]:
# Run and print result (better print format)
a=TP(data=transactions,s=freq, minSup=3000)
print(*a.miningResults().items(),sep="\n")

(1, [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]])
(2, [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [40, 34], [34, 29], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [60, 7], [40, 7], [29, 7], [52, 7], [58, 7], [36, 60], [40, 36], [36, 29], [36, 52], [58, 36], [40, 60], [60, 29], [60, 52], [58, 60], [40, 29], [40, 52], [40, 58], [52, 29], [58, 29], [58, 52]])
(3, [[48, 58, 52], [56, 52, 29], [56, 58, 29], [56, 58, 52], [66, 60, 29], [66, 60, 52], [66, 60, 58], [66, 52, 29], [66, 58, 29], [66, 52, 58], [40, 34, 29], [40, 34, 52], [40, 34, 58], [34, 52, 29], [34, 58, 29], [34, 52, 58], [60, 29, 62], [60, 62, 52], [58, 60, 62], [40, 29, 62], [40, 52, 62], [40, 58, 62], [52, 29, 62], [58, 29, 62], [58, 52, 62], [40, 60, 7], [60, 29, 7], [60, 52, 7], [58, 60, 7], [40, 29, 7], [40, 52, 7], [40, 58, 7], [52, 29, 7], [58, 29, 7], [58, 52, 7], [40, 36, 60], [36, 29, 60], [36, 60, 52], [58, 36, 60], [

### Answer questions here:


**Question 1. Why should we sort all items in s by it's support and why we have to restored it to new artribute s?**

**Answer:**
Lý do chúng ta phải sort tất cả item trong s theo support bởi vì chúng ta có thể tỉa sớm những node mà nó không thỏa min_sup trước tiên mà không sinh ra các nhánh con từ node đó. Ví dụ nếu như ta không sort item theo support, giả sử ta đang xét node A thỏa mãn min_sup và tồn tại một node B phía sau node A thì theo thuật toán ta vẫn sẽ tạo ra nhánh A->B mặc dù nhánh này không thỏa min_sup và nhánh này chỉ được tỉa khi xét tới level tiếp theo của node A. Chính vì thế ta sort item theo support sẽ loại bỏ được trường hợp này mà không tạo ra nhánh mới và do đó trực tiếp tăng hiệu suất của thuật toán. Chúng ta lưu kết quả đó vào một biến s khác để dễ dàng cho việc sắp xếp và khởi tạo theo thứ tự support ta mong muốn ở bước initialize.


**Question 2. Python set does not remain elements order so we use a list to extend it easily when create new itemset but why we store itemset in data by a python set????**

**Answer:**
Bởi vì set trong python có nhiều ưu điểm hỗ trợ tốt hơn trong bài lab này. Thứ quan trọng nhất của set so với list là khả năng truy cập thông qua các key mà không cần index cụ thể như list. Nó giúp chúng ta dễ dàng tính được giá trị support cho tập itemset hơn so với khi dùng list để lưu giá trị. Hơn nữa set không cho lưu các giá trị lặp lại nên đem lại thêm 1 ưu điểm khác là giúp chúng ta dễ dàng join 2 tập itemset từ level k cho ra level k+1.


**Question 3.  After finish implementing the algorithm tell me why should you use this instead of delete item directly from search_space and tree.**

**Answer:**
Khi một itemset bị xóa trực tiếp khỏi tree và search_space có thể dẫn đến việc không gặp phải node đó khi duyệt qua cây và khiến cho chúng ta có tiềm tàng khả năng mất mát một số phần hữu ích trên cây

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). You can download dataset from here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/churn.txt. Write your report and implementation below.

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  
More information about `churn` here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir4/Churn.pdf)*

**TODO:**
Chúng ta sẽ biến đổi transactinoal database thành flat file. Ý tưởng được đưa ra để phân biệt được các item trong 1 tập itemset giữa các transaction là ta sẽ biến đổi từng item thành dạng {header}_{item} và thực hiện việc lấy các tập frequent itemset như thuật toán đã cài đặt phía trên.

In [6]:

def readDataWoHeader(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) #* Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        headers = f.readline()
        headers = list(map(str, headers.strip().split(',')))
        tid=1
        for line in f:
            itemset=list(map(str,line.strip().split(','))) #Using list to keep order itemset
            for header, item in zip(headers, itemset):  
                s[f'{header}_{item}']+=1     #* Why don't we compute support of items while reading data?
            data[tid]= set(f'{header}_{item}' for header, item in zip(headers, itemset))
            tid+=1
    return data, s

In [7]:
churnTransactions, churnFreq= readDataWoHeader('churn.txt')

In [8]:
# Run and print result (better print format)
a=TP(data=churnTransactions,s=churnFreq, minSup=500)
print(*a.miningResults().items(),sep="\n")

(1, [['Intl Calls_4'], ['Intl Calls_3'], ['CustServ Calls_0'], ['CustServ Calls_2'], ['Area Code_408'], ['Area Code_510'], ['VMail Plan_yes'], ['CustServ Calls_1'], ['Area Code_415'], ['VMail Plan_no'], ['VMail Message_0'], ['Churn?_False.'], ["Int'l Plan_no"]])
(2, [['Churn?_False.', 'Intl Calls_4'], ["Int'l Plan_no", 'Intl Calls_4'], ['Churn?_False.', 'Intl Calls_3'], ["Int'l Plan_no", 'Intl Calls_3'], ['CustServ Calls_0', 'VMail Plan_no'], ['CustServ Calls_0', 'VMail Message_0'], ['CustServ Calls_0', 'Churn?_False.'], ['CustServ Calls_0', "Int'l Plan_no"], ['CustServ Calls_2', 'VMail Plan_no'], ['CustServ Calls_2', 'VMail Message_0'], ['CustServ Calls_2', 'Churn?_False.'], ['CustServ Calls_2', "Int'l Plan_no"], ['Area Code_408', 'VMail Plan_no'], ['Area Code_408', 'VMail Message_0'], ['Area Code_408', 'Churn?_False.'], ['Area Code_408', "Int'l Plan_no"], ['VMail Plan_no', 'Area Code_510'], ['VMail Message_0', 'Area Code_510'], ['Churn?_False.', 'Area Code_510'], ["Int'l Plan_no", 'A

# 4 References

http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/ProjectDefinition1.pdf

https://cs.wmich.edu/~alfuqaha/summer14/cs6530/lectures/AssociationAnalysis-Part1.pdf

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
